In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import periodogram, medfilt2d
from scipy.interpolate import interp1d
from scipy.fftpack import next_fast_len

In [ ]:
import matplotlib.pyplot as plt
# %matplotlib notebook
figsize=[19 , 5]
isPlot = False

In [ ]:
maxFreq = 60
maxHarmonic = 50
peddingFactor = 100
detectionThresh = 2

In [ ]:
t = pd.read_csv('Hwp - motor - bearing 1- 12_16_16 - 18_12.csv').iloc[:, 0].values

x = pd.concat([
    pd.read_csv('Hwp - motor - bearing 1- 12_16_16 - 18_12.csv').iloc[:, 1:],
    pd.read_csv('Hwp - motor - bearing 2- 12_16_16 - 18_12.csv').iloc[:, 1:],
              ], axis=1).values
x.shape

In [ ]:
# first phase for white signal estimation:
nfft = next_fast_len(x.shape[0])
f, pxx = periodogram(x, fs=1 / t[1], return_onesided=True, axis=0, nfft=nfft)
cond = f < maxFreq * maxHarmonic * 1.1
pxx = pxx[cond]
f = f[cond]

winSize = (np.round(10/ f[1] / 2) * 2).astype(int) + 1
pxxMed = medfilt2d(np.log(pxx), (winSize, 1))
interpFuns = [interp1d(f, pxxMed[:, channel]) for channel in range(pxxMed.shape[1])]

# upsampled spectrum for later HPS with better resolution:
nfft = next_fast_len(x.shape[0]) * peddingFactor
f, pxx = periodogram(x, fs=1 / t[1], return_onesided=True, axis=0, nfft=nfft)
cond = f < maxFreq * maxHarmonic
pxx = pxx[cond]
f = f[cond]

# pre-whitening section:
pxxMed = np.concatenate([np.atleast_2d(interpFun(f)) for interpFun in interpFuns], axis=0).T
logPxxDetrand = np.log(pxx) - pxxMed

# thresholding
logPxxDetrand[logPxxDetrand < detectionThresh] = 0
logPxxDetrand += 1
# channel product
probAxisProd = np.prod(logPxxDetrand, axis=1)

# Harmonic Product spectrum implementation:
nSearch = (f < maxFreq).sum()
probHarmonicProd = np.ones((nSearch, ))
maxLoc = [] #monitoring variable
snr = [] #monitoring variable
for harmonic in range(1, maxHarmonic + 1):
    probHarmonicProd *= probAxisProd[:nSearch * harmonic:harmonic]
    
    # monitoring and visualization:
    maxLoc.append(np.argmax(probHarmonicProd))
    snr.append(probHarmonicProd[maxLoc[-1]] / probHarmonicProd.mean())
    if isPlot:
        plt.figure(figsize=figsize)
        plt.title("Harmonic No. {harmonic}".format(harmonic=harmonic))
        plt.semilogy(f[:nSearch], probHarmonicProd)
        plt.semilogy(f[:nSearch], probHarmonicProd / probAxisProd[:nSearch * harmonic:harmonic])
        plt.legend(['after', 'before'])
        plt.grid()
        plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(f[maxLoc] - f[maxLoc[-1]])
plt.ylabel('$\Delta$ f [Hz]')
plt.xlabel('Harmonic')
plt.ylim([-0.025, 0.025])
plt.grid()
plt.show()

plt.figure(figsize=figsize)
plt.plot(snr)
plt.xlabel('Harmonic')
plt.grid()
plt.ylabel('frac{Max Peak}{Mean value}')
plt.show()